In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt

# Turn interactive plotting off
plt.ioff()

HOME = 0
WORK = 1

HEALTHY = 0
INFECTED = 1
TRANSMITTER = 2

def calc_coords_in_circle(x, y, radius, x_size, y_size):
    """ Calculate coordinates in circle that satisfy radius & grid (must be located within) conditions
    
    :param x:      Current x coordinate in a grid
    :param y:      Current y coordinate in a grid
    :param radius: Desired radius of a circle
    :param x_size: Grid x size (of map to place people into)
    :param y_size: Grid y size (of map to place people into)
    :return:       Satisfying coordinates
    """
    
    coords = []

    # Possible y-shifts
    for i in range(-int(np.floor(radius)), int(np.floor(radius)) + 1):
        
        # Possible x-shifts
        for j in range(int(np.floor(radius)) + 1):
            
            # Coords in circle
            if np.sqrt(i ** 2 + j ** 2) <= radius:
                
                # Append valid coordinates
                if (x + j) >= 0 and (y + i) >= 0 and (x + j) < x_size and (y + i) < y_size:
                    coords.append((x + j, y + i))
                
                # Append valid mirrored coordinates
                if j > 0:
                    
                    if (x - j) >= 0 and (y + i) >= 0 and (x - j) < x_size and (y + i) < y_size:
                        coords.append((x - j, y + i))
                
            # Coords not in circle
            else:
                
                break

    return np.array(coords)


def walk_iter(cities_list, radius, neighbourhood_radius):
    """ Walk people near their home

    :param cities_list:          List of CityResidents class objects
    :param radius:               Limiting radius for one epoch walk
    :param neighbourhood_radius: Maximum distance allowed to travel for each person from his home location
    """
    
    for city in cities_list:
    
        for i, (x, y, location) in enumerate(zip(city.cur_x_arr, city.cur_y_arr, city.location_arr)):
    
            # Skip working person
            if location == WORK:
                continue

            valid_coords = False

            # Get possible moves from current (x, y) location
            coords = calc_coords_in_circle(x, y, radius, city.x_size, city.y_size)

            # Randomly choose new move that yields neighbourhood coordinates
            while not valid_coords:

                rnd_move_idx = np.random.choice(coords.shape[0], 1, replace=False)[0]
                new_x, new_y = coords[rnd_move_idx]
                if np.sqrt((new_x - city.home_x_arr[i]) ** 2 + (new_y - city.home_y_arr[i]) ** 2) <= neighbourhood_radius:
                    valid_coords = True

            city.cur_x_arr[i] = new_x
            city.cur_y_arr[i] = new_y

        
def make_disease_matrix(cities_list, city_idx, spread_radius):
    """ Make disease (exposure) matrix which indicates for each spot a number of ill people near, that can transfer a disease
    
    :param cities_list:   List of CityResidents class objects
    :param city_idx:      Index of city of interest in cities_list
    :param spread_radius: Radius of exposure of an infected person within which disease can be transfered to a healthy one
    :return:              Disease (exposure) matrix
    """

    city = cities_list[city_idx]

    # Init disease matrix
    disease_mat = np.zeros(shape=(city.y_size, city.x_size)).astype(int)

    # Get indices of residents with trasmittable disease
    disease_indices = np.where((city.location_arr == HOME) & (city.status_arr == TRANSMITTER))[0]

    # Calculate disease spread for each infected person
    for i in disease_indices:

        infected_coords = calc_coords_in_circle(city.cur_x_arr[i], city.cur_y_arr[i], spread_radius, city.x_size, city.y_size)

        for x, y in infected_coords:
            disease_mat[y, x] += 1

    # Process workers from other cities with trasmittable disease
    for n in range(len(cities_list)):
    
        # Skip current city
        if n == city_idx:
            continue
    
        # Get indices of workers from other cities with trasmittable disease
        disease_indices = np.where((cities_list[n].work_city_arr == city_idx) & (cities_list[n].location_arr == WORK) & (cities_list[n].status_arr == TRANSMITTER))[0]

        # Calculate disease spread for each infected person
        for i in disease_indices:

            infected_coords = calc_coords_in_circle(cities_list[n].work_x_arr[i], cities_list[n].work_y_arr[i], 
                                                    spread_radius, city.x_size, city.y_size)

            for x, y in infected_coords:
                disease_mat[y, x] += 1
                
    return disease_mat


def make_disease_matrices(cities_list, spread_radius):
    """ Make disease (exposure) matrices for each city
    
    :param cities_list:   List of CityResidents class objects
    :param spread_radius: Radius of exposure of an infected person within which disease can be transfered to a healthy one
    :return:              List of disease (exposure) matrices
    """
    
    disease_mat_list = []
    
    for city_idx in range(len(cities_list)):
        
        disease_mat = make_disease_matrix(cities_list, city_idx, spread_radius)
        disease_mat_list.append(disease_mat)
        
    return disease_mat_list


def calc_infection_prob(infection_exposure, infect_prob = 0.5):
    """ Calculate infection probability based on exposure and infection's base probability
    
    :param infection_exposure: Number of infected people which can transfer the disease and located near a healthy person
    :param infect_prob:        Infection's base probability
    :return:                   Probability of infection
    """
    return 1 - (1 - infect_prob) ** infection_exposure


def spread_disease(disease_mat_list, cities_list, timer_min, timer_max, infect_prob):
    """ Spread disease between infected and healthy people based on disease_mat_list. Updates status_arr & incubation_timer_arr in cities_list objects
    
    :param disease_mat_list: List of maps of overlapping areas of disease exposure from current infected people
    :param cities_list:      List of CityResidents class objects
    :param timer_min:        Min steps (epochs) until infected person can transmit a disease (exception: initial group)
    :param timer_max:        Max steps (epochs) until infected person can transmit a disease (exception: initial group)
    :param infect_prob:      Disease's base probability of transmission
    """

    for city_idx, (city, disease_mat) in enumerate(zip(cities_list, disease_mat_list)):

        # Get indices of healthy residents
        disease_indices = np.where((city.location_arr == HOME) & (city.status_arr == HEALTHY))[0]
        
        # Transmit disease between residents in city
        for i in disease_indices:
            
            infection_exposure = disease_mat[city.cur_y_arr[i], city.cur_x_arr[i]]
        
            # Calculate probability of getting ill and illness' outcome
            infection_prob = calc_infection_prob(infection_exposure, infect_prob)
            infection_outcome = np.random.choice([HEALTHY, INFECTED], p=[1 - infection_prob, infection_prob], replace=False)
        
            # Infect only healthy people
            assert city.status_arr[i] == HEALTHY
            if infection_outcome == INFECTED and city.incubation_timer_arr[i] == 0:
                city.status_arr[i] = infection_outcome
                city.incubation_timer_arr[i] = np.random.choice(range(timer_min, timer_max + 1), replace=False)

                # Instant transmitter
                if city.incubation_timer_arr[i] == 0:
                    city.status_arr[i] = TRANSMITTER 
                    
        # Transmit disease between other workers in city
        for n in range(len(cities_list)):
    
            # Skip current city
            if n == city_idx:
                continue

            # Get indices of healthy workers from other cities
            disease_indices = np.where((cities_list[n].work_city_arr == city_idx) & (cities_list[n].location_arr == WORK) & (cities_list[n].status_arr == HEALTHY))[0]
            
            # Transmit disease between other workers in city
            for i in disease_indices:

                infection_exposure = disease_mat[cities_list[n].work_y_arr[i], cities_list[n].work_x_arr[i]]

                # Calculate probability of getting ill and illness' outcome
                infection_prob = calc_infection_prob(infection_exposure, infect_prob)
                infection_outcome = np.random.choice([HEALTHY, INFECTED], p=[1 - infection_prob, infection_prob], replace=False)

                # Infect only healthy people
                assert cities_list[n].status_arr[i] == HEALTHY
                if infection_outcome == INFECTED and cities_list[n].incubation_timer_arr[i] == 0:
                    cities_list[n].status_arr[i] = infection_outcome
                    cities_list[n].incubation_timer_arr[i] = np.random.choice(range(timer_min, timer_max + 1), replace=False)

                    # Instant transmitter
                    if cities_list[n].incubation_timer_arr[i] == 0:
                        cities_list[n].status_arr[i] = TRANSMITTER

            
def plot_disease_exposure(cities_list, city_idx, spread_radius, epoch=None, path=None, ax=None):
    """ Plot disease (exposure) matrix for one city
    
    :param cities_list:   List of CityResidents class objects
    :param city_idx:      Index of city of interest in cities_list
    :param spread_radius: Disease spreading radius
    :param epoch:         Current epoch in simulation
    :param path:          Absolute path to save plot to
    :param ax:            Axis to plot
    """
    
    city = cities_list[city_idx]
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(15, 15))

    
    # Plot all residents that are in that city
    home_indices = np.where(city.location_arr == HOME)[0]
    ax.scatter(city.cur_x_arr[home_indices], city.cur_y_arr[home_indices], s=1, c='grey', alpha=0.1, label='Uninfected')

    # Plot all other workers in that city
    for n in range(len(cities_list)):
    
        # Skip current city
        if n == city_idx:
            continue
        
        work_indices = np.where((cities_list[n].work_city_arr == city_idx) & (cities_list[n].location_arr == WORK))[0]
        ax.scatter(cities_list[n].work_x_arr[work_indices], cities_list[n].work_y_arr[work_indices], s=1, c='grey', alpha=0.1)
        
        
    # Plot all infected residents that are in that city
    ax.scatter(city.cur_x_arr[(city.status_arr == INFECTED) & (city.location_arr == HOME)], 
               city.cur_y_arr[(city.status_arr == INFECTED) & (city.location_arr == HOME)], s=spread_radius, c='yellow', alpha=0.5, label='Infected (visible + invisible)')
    
    # Plot all other infected workers in that city
    for n in range(len(cities_list)):
    
        # Skip current city
        if n == city_idx:
            continue
        
        work_indices = np.where((cities_list[n].work_city_arr == city_idx) & (cities_list[n].location_arr == WORK) & (cities_list[n].status_arr == INFECTED))[0]
        ax.scatter(cities_list[n].work_x_arr[work_indices], cities_list[n].work_y_arr[work_indices], s=spread_radius, c='yellow', alpha=0.5)
    
    
    # Plot all residents-transmitters that are in that city
    ax.scatter(city.cur_x_arr[(city.status_arr == TRANSMITTER) & (city.location_arr == HOME)], 
               city.cur_y_arr[(city.status_arr == TRANSMITTER) & (city.location_arr == HOME)], s=spread_radius, c='red', alpha=1, label='Infected (visible = transmitters)')
    
    # Plot all other workers-transmitters in that city
    for n in range(len(cities_list)):
    
        # Skip current city
        if n == city_idx:
            continue
        
        work_indices = np.where((cities_list[n].work_city_arr == city_idx) & (cities_list[n].location_arr == WORK) & (cities_list[n].status_arr == TRANSMITTER))[0]
        ax.scatter(cities_list[n].work_x_arr[work_indices], cities_list[n].work_y_arr[work_indices], s=spread_radius, c='red', alpha=1)
        
        
    ax.set_title(f'City {city_idx}={city.city_code}. Disease (exposure) matrix - visibly ill people (in a city) = {(city.status_arr == TRANSMITTER).sum()}, infected total = {(city.status_arr != HEALTHY).sum()} - epoch {epoch}')
    ax.legend(loc=2)

    if ax is None:
    
        if epoch is not None and path is not None:

            # Create folder if it doesn't exist
            if not os.path.exists(path):
                os.mkdir(path)

            # Save plot
            fig.savefig(os.path.join(path, f'City_{city_idx}_{city.city_code}_disease_matrix_epoch_{epoch}.png'), dpi=300)

        plt.close(fig)
    
    
def plot_disease_exposures(cities_list, spread_radius, epoch=None, path=None):
    """ Plot disease (exposure) matrix for one city
    
    :param cities_list:   List of CityResidents class objects
    :param spread_radius: Disease spreading radius
    :param epoch:         Current epoch in simulation
    :param path:          Absolute path to save plot to
    """
    
    plot_dict = {
        0: {'x': 1, 'y': 1},
        1: {'x': 1, 'y': 0},
        2: {'x': 0, 'y': 1},
        3: {'x': 0, 'y': 2},
        4: {'x': 2, 'y': 3},
        5: {'x': 1, 'y': 3},
        6: {'x': 3, 'y': 2},
        7: {'x': 3, 'y': 1},
        8: {'x': 3, 'y': 0},
        9: {'x': 2, 'y': 0}
    }
    
    
    fig, ax = plt.subplots(ncols=4, nrows=4, figsize=(4 * 15, 4 * 15))
    
    for city_idx in range(len(cities_list)):
        plot_disease_exposure(cities_list, city_idx, spread_radius, epoch=epoch, path=path, 
                              ax=ax[plot_dict[city_idx]['y'], plot_dict[city_idx]['x']])
    
    if epoch is not None and path is not None:

        # Create folder if it doesn't exist
        if not os.path.exists(path):
            os.mkdir(path)

        # Save plot
        fig.savefig(os.path.join(path, f'All_cities_disease_matrix_epoch_{epoch}.png'), dpi=300)

    plt.close(fig)
    

class CityResidents:
    
    def __init__(self, city_num, city_code, x_size, y_size, residents_num, init_transmitters_num):
        self.city_num = city_num
        self.city_code = city_code
        self.x_size = x_size
        self.y_size = y_size
        self.residents_num = residents_num
        
        self.init_home_coords()
        self.init_current_location()
        self.init_work_location()
        self.init_disease_arrays()
        self.init_infected_group(init_transmitters_num)
        
    def init_home_coords(self):
        self.home_x_arr = np.random.randint(self.x_size, size=self.residents_num)
        self.home_y_arr = np.random.randint(self.y_size, size=self.residents_num)
        
    def init_current_location(self):
        self.cur_x_arr = self.home_x_arr.copy()
        self.cur_y_arr = self.home_y_arr.copy()
        self.location_arr = np.zeros(self.residents_num).astype(int)
        
    def init_disease_arrays(self):
        self.status_arr = np.zeros(self.residents_num).astype(int)            # Illness status
        self.incubation_timer_arr = np.zeros(self.residents_num).astype(int)  # Timer untill illnes becomes observable (& transmittable)
        
    def init_infected_group(self, init_transmitters_num):
        self.status_arr[np.random.choice(self.residents_num, init_transmitters_num, replace=False)] = TRANSMITTER
    
    def init_work_location(self):
        
        # Init working place as a city of residence
        self.work_city_arr = np.array(self.residents_num * [self.city_num])
        self.work_x_arr = np.random.randint(self.x_size, size=self.residents_num)
        self.work_y_arr = np.random.randint(self.y_size, size=self.residents_num)
        
    def add_work_location(self, city_num, x_size, y_size, workers_num):
        
        # Randomly select 'workers_num' who will work in other city
        indices = np.random.choice(np.where(self.work_city_arr == self.city_num)[0], workers_num, replace=False)
        self.work_x_arr[indices] = np.random.randint(x_size, size=workers_num)
        self.work_y_arr[indices] = np.random.randint(y_size, size=workers_num)
        self.work_city_arr[indices] = city_num


def decrement_incubation_timer(cities_list):
    """ Decrement incubation timer across cities
    
    :param cities_list: List of CityResidents class objects
    """
    
    for city in cities_list:
    
        # Make illness observable
        new_disease_observations = np.where(city.incubation_timer_arr == 1)[0]
        city.status_arr[new_disease_observations] = TRANSMITTER

        # Decrement people counter with unabservable illness
        disease_indices = np.where(city.incubation_timer_arr > 0)[0]
        city.incubation_timer_arr[disease_indices] -= 1
     
    
def transport_to_work(cities_list, amount='third'):
    """ Transport people to work across cities
    
    :param cities_list: List of CityResidents class objects
    """
    
    for city in cities_list:
    
        if amount == 'third':
            worker_indices = np.random.choice(np.where(city.location_arr == HOME)[0], int(np.floor(city.residents_num / 3)), replace=False)
            
        elif amount == 'rest':
            worker_indices = np.where(city.location_arr == HOME)[0]
            
        else:
            print('[transport_to_work] amount value is undefined !')
            
        city.location_arr[worker_indices] = WORK
        
        
def transport_to_home(cities_list, amount='third'):
    """ Transport people to home across cities

    :param cities_list: List of CityResidents class objects
    """
    
    for city in cities_list:
    
        if amount == 'third':
            worker_indices = np.random.choice(np.where(city.location_arr == WORK)[0], int(np.floor(city.residents_num / 3)), replace=False)
            
        elif amount == 'rest':
            worker_indices = np.where(city.location_arr == WORK)[0]
            
        else:
            print('[transport_to_home] amount value is undefined !')
            
        city.location_arr[worker_indices] = HOME
        
        # Reset coordinates to home
        city.cur_x_arr[worker_indices] = city.home_x_arr[worker_indices]
        city.cur_y_arr[worker_indices] = city.home_y_arr[worker_indices]
        
        
def track_stats(cities_list, disease_tracker, visible_disease_tracker):
    """ Track infected & transmitters
    
    :param cities_list:             List of CityResidents class objects
    :param disease_tracker:         List of infected + transmitters num for previous epochs
    :param visible_disease_tracker: List of transmitters num for previous epochs
    """
    
    infected = 0
    transmitters = 0
    
    for city in cities_list:
        
        infected += (city.status_arr != HEALTHY).sum()
        transmitters += (city.status_arr == TRANSMITTER).sum()
        
    disease_tracker.append(infected)
    visible_disease_tracker.append(transmitters)
    
    
def simulate_transportations_with_infections(init_transmitters_num, timer_min, timer_max, neighbourhood_radius, infect_prob, radius, spread_radius, epochs, plot_disease_matrix=None):
    """ Simulate people transportation and disease spread in a square grid
    
    :param init_transmitters_num: Initial infected people number
    :param timer_min:             Min steps (epochs) until infected person can transmit a disease (exception: initial group)
    :param timer_max:             Max steps (epochs) until infected person can transmit a disease (exception: initial group)
    :param neighbourhood_radius:  Maximum distance allowed to travel for each person from his initial location
    :param epochs:                Steps to perform during each people 1) travel and 2) spread the disease
    :param radius:                Maximum radius for person to travel in single epoch
    :param spread_radius:         Disease spreading radius
    :param infect_prob:           Base probability for disease to transmit
    :param plot_disease_matrix:   Path to save plot of disease (exposure) matrix before transmitting a disease in each epoch
    :return:                      Number of ill (visible + invisible) people for each epoch, number of ill (visible) people that can transmit a disease for each epoch
    """

    ###########################
    # Init variables          #
    ###########################

    cities_list = []

    msk = CityResidents(city_num=0, city_code='msk', x_size=506, y_size=506, residents_num=126781, init_transmitters_num=init_transmitters_num)
    khi = CityResidents(city_num=1, city_code='khi', x_size=105, y_size=105, residents_num=2596,   init_transmitters_num=0)
    kra = CityResidents(city_num=2, city_code='kra', x_size=51,  y_size=51,  residents_num=1756,   init_transmitters_num=0)
    odi = CityResidents(city_num=3, city_code='odi', x_size=44,  y_size=44,  residents_num=1355,   init_transmitters_num=0)
    dom = CityResidents(city_num=4, city_code='dom', x_size=126, y_size=126, residents_num=1372,   init_transmitters_num=0)
    pod = CityResidents(city_num=5, city_code='pod', x_size=64,  y_size=64,  residents_num=3081,   init_transmitters_num=0)
    lub = CityResidents(city_num=6, city_code='lub', x_size=36,  y_size=36,  residents_num=2053,   init_transmitters_num=0)
    sho = CityResidents(city_num=7, city_code='sho', x_size=72,  y_size=72,  residents_num=1261,   init_transmitters_num=0)
    bal = CityResidents(city_num=8, city_code='bal', x_size=79,  y_size=79,  residents_num=5074,   init_transmitters_num=0)
    myt = CityResidents(city_num=9, city_code='myt', x_size=59,  y_size=59,  residents_num=2355,   init_transmitters_num=0)

    cities_list.append(msk)
    cities_list.append(khi)
    cities_list.append(kra)
    cities_list.append(odi)
    cities_list.append(dom)
    cities_list.append(pod)
    cities_list.append(lub)
    cities_list.append(sho)
    cities_list.append(bal)
    cities_list.append(myt)

    # Add work location for msk residents
    msk.add_work_location(city_num=khi.city_num, x_size=khi.x_size, y_size=khi.y_size, workers_num=42)
    msk.add_work_location(city_num=kra.city_num, x_size=kra.x_size, y_size=kra.y_size, workers_num=29)
    msk.add_work_location(city_num=odi.city_num, x_size=odi.x_size, y_size=odi.y_size, workers_num=22)
    msk.add_work_location(city_num=dom.city_num, x_size=dom.x_size, y_size=dom.y_size, workers_num=22)
    msk.add_work_location(city_num=pod.city_num, x_size=pod.x_size, y_size=pod.y_size, workers_num=50)
    msk.add_work_location(city_num=lub.city_num, x_size=lub.x_size, y_size=lub.y_size, workers_num=33)
    msk.add_work_location(city_num=sho.city_num, x_size=sho.x_size, y_size=sho.y_size, workers_num=20)
    msk.add_work_location(city_num=bal.city_num, x_size=bal.x_size, y_size=bal.y_size, workers_num=82)
    msk.add_work_location(city_num=myt.city_num, x_size=myt.x_size, y_size=myt.y_size, workers_num=38)

    # Add work location for residents of other cities
    khi.add_work_location(city_num=msk.city_num, x_size=msk.x_size, y_size=msk.y_size, workers_num=695)
    kra.add_work_location(city_num=msk.city_num, x_size=msk.x_size, y_size=msk.y_size, workers_num=470)
    odi.add_work_location(city_num=msk.city_num, x_size=msk.x_size, y_size=msk.y_size, workers_num=363)
    dom.add_work_location(city_num=msk.city_num, x_size=msk.x_size, y_size=msk.y_size, workers_num=367)
    pod.add_work_location(city_num=msk.city_num, x_size=msk.x_size, y_size=msk.y_size, workers_num=825)
    lub.add_work_location(city_num=msk.city_num, x_size=msk.x_size, y_size=msk.y_size, workers_num=550)
    sho.add_work_location(city_num=msk.city_num, x_size=msk.x_size, y_size=msk.y_size, workers_num=338)
    bal.add_work_location(city_num=msk.city_num, x_size=msk.x_size, y_size=msk.y_size, workers_num=1359)
    myt.add_work_location(city_num=msk.city_num, x_size=msk.x_size, y_size=msk.y_size, workers_num=631)
    
    ###########################
    # Run simulations         #
    ###########################

    disease_tracker = []
    visible_disease_tracker = []
    
    hour = 1
    
    for i in tqdm(range(1, epochs + 1)):

        if hour > 24:
            hour -= 24
        
        # Make disease visible (and transmittable)
        if i > 1:
            decrement_incubation_timer(cities_list)
            
        # Transport one-third of the people to work
        if (hour % 7 == 0) & (hour % 2 != 0) & (hour % 3 != 0):
            transport_to_work(cities_list, amount='third')
            
        # Transport one-third of the people to work
        elif (hour % 8 == 0) & (hour % 2 != 0) & (hour % 3 != 0):
            transport_to_work(cities_list, amount='third')
            
        # Transport rest (one-third) of the people to work
        elif (hour % 9 == 0) & (hour % 2 != 0):
            transport_to_work(cities_list, amount='rest')
            
        # Transport one-third of the people from work
        elif hour % 19 == 0:
            transport_to_home(cities_list, amount='third')
            
        # Transport one-third of the people from work
        elif hour % 20 == 0:
            transport_to_home(cities_list, amount='third')
            
        # Transport rest (one-third) of the people from work
        elif hour % 21 == 0:
            transport_to_home(cities_list, amount='rest')
            
        # Walk peaple that are near their home
        walk_iter(cities_list, radius, neighbourhood_radius)
        
        # Observe disease maps
        disease_mat_list = make_disease_matrices(cities_list, spread_radius)
        
        # Plot & save disease exposure map
        if plot_disease_matrix is not None:
            
            plot_disease_exposures(cities_list, spread_radius, epoch=i, path=plot_disease_matrix)
            
            # for city_idx in range(len(cities_list)):
            #    plot_disease_exposure(cities_list, city_idx, spread_radius, epoch=i, path=plot_disease_matrix)
            
        # Spread disease (based on the maps above)
        spread_disease(disease_mat_list, cities_list, timer_min, timer_max, infect_prob)
        
        # Track stats
        track_stats(cities_list, disease_tracker, visible_disease_tracker)
        
        hour += 1
        
        # Debug
        print('[epoch={}]\tinfected(total)={}\ttransmitters(visibly ill)={}'.format(i, disease_tracker[-1], visible_disease_tracker[-1]))
        
    return np.array(disease_tracker), np.array(visible_disease_tracker)

In [ ]:
init_transmitters_num = 100  # Initial infected people number
timer_min = 1                # Min steps (epochs) until infected person can transmit a disease (exception: initial group)
timer_max = 12               # Max steps (epochs) until infected person can transmit a disease (exception: initial group)
neighbourhood_radius = 120   # Maximum distance allowed to travel for each person from his initial location
epochs = 120                 # Steps to perform during each people 1) travel and 2) spread the disease

radius = 1.5                 # Maximum radius for person to travel in single epoch
spread_radius = 20           # Disease spreading radius
infect_prob = 1.0            # Base probability for disease to transmit

# Path to plot of disease (exposure) matrix before transmitting a disease in each epoch
plot_disease_matrix = r'C:\Users\va\Transport_networks\radius_{}_spread_radius_{}_infected_prob_{}'.format(str(radius).replace('.', '_'),
                                                                                                           str(spread_radius).replace('.', '_'),
                                                                                                           str(infect_prob).replace('.', '_'))

# Function to perform simulation
disease_tracker, visible_disease_tracker = simulate_transportations_with_infections(init_transmitters_num, \
                                                                                    timer_min, \
                                                                                    timer_max, \
                                                                                    neighbourhood_radius, \
                                                                                    infect_prob, \
                                                                                    radius, \
                                                                                    spread_radius, \
                                                                                    epochs, \
                                                                                    plot_disease_matrix)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(disease_tracker, '.-', c='tab:blue', label='All infected people')
ax.plot(visible_disease_tracker, '.-', c='tab:red', label='Visible (transmitable) infected people')
ax.set_ylabel('Infected')
ax.set_xlabel('Time')
ax.set_title(f'Infected vs. time (radius={radius}, spread_radius={spread_radius}, infect_prob={infect_prob})')
ax.grid()
ax.legend()

fig.savefig(os.path.join(plot_disease_matrix, f'Transport_networks_{x_size}x{y_size}_{people_num}_people_{epochs}_epochs_infected_ts.png'), dpi=300);

plt.show()